In [1]:
# =============================================================================
# =============================================================================
# # 문제 07 유형(DataSet_07.csv 이용)
#
# 구분자 : comma(“,”), 400 Rows, 9 Columns, UTF-8 인코딩
#
# 대학원 진학을 위하여 어떤 항목이 중요하게 영향을 미치는지
# 아래 데이터로 분석하고자 한다.

# 컬 럼 / 정 의 / Type
# Serial_No / 구분자 번호 / Double
# GRE / GRE 시험 성적 / Double
# TOEFL / TOEFL 시험 성적 / Double
# University_Rating / 대학 평가 그룹 (1 ~ 5) / Double
# SOP / 자기 소개서 점수 (1 ~ 5) / Double
# LOR / 추천서 점수 (1 ~ 5) / Double
# CGPA / 학부 평량 평점 (10점 만점 환산 점수) / Double
# Research / 연구 참여 경험 여부 (0 / 1) / Double
# Chance_of_Admit / 합격 가능성 / Double
# =============================================================================
# =============================================================================

# =============================================================================
# (참고)
# #1
# import pandas as pd
# #2
# import scipy.stats as stats
# #3
# from sklearn.linear_model import LogisticRegression
# Solver = ‘liblinear’, random_state = 12
# =============================================================================

In [3]:
import pandas as pd

dataset7 = pd.read_csv('Dataset/Dataset_07.csv')

In [2]:
# =============================================================================
# 1. 합격 가능성에 GRE, TOEFL, CGPA 점수 가운데 가장 영향이 큰 것이 어떤 점수인지
# 알아 보기 위해서 상관 분석을 수행한다.
# - 피어슨(Pearson) 상관계수 값을 구한다.
# - Chance_of_Admit와의 가장 큰 상관계수 값을 가지는 항목의 상관계수를 소수점 넷째
# 자리에서 반올림하여 셋째 자리까지 기술하시오. (답안 예시) 0.123
# =============================================================================

In [5]:
q1 = dataset7.copy()

In [10]:
q1 = q1[['GRE', 'TOEFL', 'CGPA', 'Chance_of_Admit']].corr().Chance_of_Admit.iloc[:-1]

In [22]:
print('정답 : ',q1.sort_values(ascending=False)[0].round(3))


정답 :  0.873


In [23]:
# =============================================================================
# 2.GRE 점수의 평균 이상을 받은 그룹과 평균 미만을 받은 그룹의 CGPA 평균은 차이가
# 있는지
# 검정을 하고자 한다.
# - 적절한 검정 방법을 선택하고 양측 검정을 수행하시오 (등분산으로 가정)
# - 검정 결과, 검정통계량의 추정치를 소수점 셋째 자리에서 반올림하여 소수점 두 자리까지
# 기술하시오.
# (답안 예시) 1.23
# =============================================================================

In [24]:
q2 = dataset7.copy()

In [63]:
# GRE 평균
GRE_avg = q2.GRE.mean()

# 조건식
q2['GRE_gr'] = 'under'
q2.loc[ q2['GRE'] > GRE_avg , 'GRE_gr'] = 'over'

from scipy.stats import ttest_ind

a=q2[q2['GRE_gr'] == 'over']['CGPA']
b=q2[q2['GRE_gr'] == 'under']['CGPA']

ttest_ind(a,b, equal_var=True) # 등분산으로 가정(default = True)

## statistic이 검정 통계량
# 정답 : 19.44

Ttest_indResult(statistic=19.443291692470982, pvalue=1.1127037052806161e-59)

In [26]:
# =============================================================================
# 3.Chance_of_Admit 확률이 0.5를 초과하면 합격으로, 이하이면 불합격으로 구분하고
# 로지스틱 회귀분석을 수행하시오.
# - 원데이터만 사용하고, 원데이터 가운데 Serial_No와 Label은 모형에서 제외
# - 각 설정값은 다음과 같이 지정하고, 언급되지 않은 사항은 기본 설정값을 사용하시오
# Seed : 123
# - 로지스틱 회귀분석 수행 결과에서 로지스틱 회귀계수의 절대값이 가장 큰 변수와 그 값을
# 기술하시오. 
# (로지스틱 회귀계수는 반올림하여 소수점 둘째 자리까지 / Intercept는 제외) # 결과값에서 intercept를 제거
# (답안 예시) abc, 0.12
# =============================================================================

In [31]:
q3 = dataset7.copy().drop(['Serial_No'], axis=1)

In [39]:
import numpy as np

In [41]:
q3.Chance_of_Admit = np.where(q3.Chance_of_Admit > 0.5, 1, 0)

In [83]:
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀를 쓸 때 범주형 변수가 있는지 확인!
# Solver = ‘liblinear’, random_state = 12
model1 = LogisticRegression(random_state=12, solver='liblinear', C=100000).fit(
                                q3.drop(columns='Chance_of_Admit'),
                                q3.Chance_of_Admit)

In [84]:
sor = list(zip(q3.drop(columns='Chance_of_Admit').columns, model1.coef_[0]))

In [85]:
ans = sorted(sor, key=lambda x : x[1], reverse=True)[0]

In [86]:
print('정답 : ',ans[0],  ans[1].round(2))

정답 :  CGPA 3.34


In [91]:
## 다른 방법
p3 = pd.DataFrame({'var' : q3.drop(columns='Chance_of_Admit').columns.values,
                  'coef' : model1.coef_.reshape(-1)})

In [93]:
p3
# 정답 : CGPA, 3.339499

,var,coef
0,GRE,-0.007352
1,TOEFL,0.114938
2,University_Rating,-0.439986
3,SOP,-0.518198
4,LOR,1.058020
5,CGPA,3.339499
6,Research,0.115009
